In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoConfig
import torch

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "mps"
model_name = "celinelee/bartlarge_risctoarm_cloze2048"

In [3]:
bart_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")
bart_tokenizer.model_max_length = 2048

bart_config = AutoConfig.from_pretrained("facebook/bart-large")
bart_config.vocab_size = len(bart_tokenizer)
bart_config.max_position_embeddings = 2048

bart_model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    config=bart_config
)

/opt/homebrew/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/homebrew/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <2BD1B165-EC09-3F68-BCE4-8FE4E70CA7E2> /opt/homebrew/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <2A8DB508-8AAF-3FF1-BDFE-9EF17CC2B482> /opt/homebrew/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [4]:
source_lang = "RISC-V"
target_lang = "ARM64"
source_code = "  .file  \"cat.c\"\n  .option pic\n  .text\n  .align  1\n  .globl  main\n  .type  main, @function\nmain:\n  addi  sp,sp,-160\n  sd  ra,152(sp)\n  sd  s0,144(sp)\n  addi  s0,sp,160\n  mv  a5,a0\n  sd  a1,-160(s0)\n  sw  a5,-148(s0)\n  la  a5,__stack_chk_guard\n  ld  a4, 0(a5)\n  sd  a4, -24(s0)\n  li  a4, 0\n  ld  a5,-160(s0)\n  addi  a5,a5,8\n  ld  a5,0(a5)\n  li  a1,0\n  mv  a0,a5\n  call  open@plt\n  mv  a5,a0\n  sw  a5,-136(s0)\n  j  .L2\n.L3:\n  lw  a4,-132(s0)\n  addi  a5,s0,-128\n  mv  a2,a4\n  mv  a1,a5\n  li  a0,1\n  call  write@plt\n.L2:\n  addi  a4,s0,-128\n  lw  a5,-136(s0)\n  li  a2,99\n  mv  a1,a4\n  mv  a0,a5\n  call  read@plt\n  mv  a5,a0\n  sw  a5,-132(s0)\n  lw  a5,-132(s0)\n  sext.w  a5,a5\n  bne  a5,zero,.L3\n  li  a0,10\n  call  putchar@plt\n  lw  a5,-136(s0)\n  mv  a0,a5\n  call  close@plt\n  li  a5,0\n  mv  a4,a5\n  la  a5,__stack_chk_guard\n  ld  a3, -24(s0)\n  ld  a5, 0(a5)\n  xor  a5, a3, a5\n  li  a3, 0\n  beq  a5,zero,.L5\n  call  __stack_chk_fail@plt\n.L5:\n  mv  a0,a4\n  ld  ra,152(sp)\n  ld  s0,144(sp)\n  addi  sp,sp,160\n  jr  ra\n  .size  main, .-main\n  .ident  \"GCC: (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0\"\n  .section  .note.GNU-stack,\"\",@progbits\n"

text = f"Convert the following {source_lang} assembly code to {target_lang} assembly:\n```{source_lang.lower()}asm\n{source_code}```"

tokens = bart_tokenizer(
    source_code,
    return_tensors="pt"
).to(device)

In [ ]:
kwargs = {
    "output_attentions": True,
    "return_dict_in_generate": True,
    "output_scores": True,
    "num_beams": 20,
    "num_return_sequences": 20
}
output = bart_model.generate(
    **tokens,
    max_new_tokens=2048,
    # temperature=0.7,
    # do_sample=True,
    **kwargs
)

/opt/homebrew/lib/python3.11/site-packages/transformers/pytorch_utils.py:338: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)
BartModel is using BartSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True` or `layer_head_mask` not None. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


In [6]:
decoded = bart_tokenizer.batch_decode(
    output.sequences,
    skip_special_tokens=True
)

In [13]:
for i, asm in enumerate(decoded):
    print(f"Variant [{i}]")
    print(asm)

Variant [0]
  .file  "cat c"
	.text
  .align    .type  .globl  main, %function
main:
   0type,  x0,sp,.   sp,  s0,160
  mv  a5,a0
  x1,-160)
  sw  x5,__stack_chk_guard
  ld  x4, [sp, 16]  addi  x2, -x0,0, 0
  0 "" 2
#NO_APP
	ldr	w1, [x0]
	mov	w0, w1
	and	x3, x0
	add	x1, x3, 524
	str	x5, [w4, 0x1]  0x0  4]  call   x3
	orr	x2, x1, 0)
.L3:
	sub	sp, sp, #128
	   lw3,s0,   4.0,s1,a4
	bl	__aarch64_ldadd4_acq_rel
	cmp	x4, 1
	beq	.L2
	is   [x4]
.align	2
.global	__v4_FUNr   a1,99
	stv2, v0.4 
	 call  read
	nop   rv   #4s0)
	 lsl #1(s0   5
	umv  #5, -8(x0)	
	scv4.4h, rv, #4(x1)

Variant [1]
  .file  "cat c"
	.text
  .align    .type  .globl  main, %function
main:
   0type,  x0,sp,.   sp,  s0,160
  mv  a5,a0
  x1,-160)
  sw  x5,__stack_chk_guard
  ld  x4, [sp, 16]  addi  x2, -x0,0, 0
  0 "" 2
#NO_APP
	ldr	w1, [x0]
	mov	w0, w1
	and	x3, x0
	add	x1, x3, 524
	str	x5, [w4, 0x1]  0x0  4]  call   x3
	orr	x2, x1, 0)
.L3:
	sub	sp, sp, #128
	   lw3,s0,   4.0,s1,a4
	bl	__aarch64_ldadd4_acq_rel
	cmp	x4, 1
	be